In [ ]:
1단원
3. 각 샘플에 대해 원하는 답을 담고있는 훈련세트
15. 훈련 데이터에 과대적합 / 파라미터 수가 적은 모델을 선택하거나, 훈련 데이터를 많이 모으거나 훈련 데이터 잡음 줄이기

In [1]:
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(housing_prepared, housing_labels)

NameError: name 'SVR' is not defined

In [ ]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse
#randomforest(22149)에 비해 오차가 커짐

In [ ]:
grid_search.best_params_
#rbf보다 linear의 성능이 더 나음